In [56]:
import pixiedust

In [57]:
import pandas as pd

In [58]:
df = pd.read_csv('operations-report-hdrvg(1).csv')

In [59]:
df = df.dropna(subset = ['VDC'], how='all')

In [60]:
df['Location name'] = df['VDC'] + ', ' + df['Village/Ward/extra info'].fillna('')
# df['Location name'].apply(lambda x: x[:-2] if x[-2:] == ', ' else x )[29]

In [61]:
df

,Items,Origin,Items grouped,Item group filter,Category,Unit,amount,"FSC - KGs (sack=25, pack=1, box=4.5)",FSC - Total KGs,Mission,District,VDC,Village/Ward/extra info,VDC Latitude,VDC Longitude,District Latitude,District Longitude,Date,Location name
0,NaN,NaN,Noodles,NaN,Food - Dry Food,box,5.0,4.5,22.5,MISSION 29/4 *01,Bhaktapur,Nankhel,Maligaun,27.64220,85.45980,27.66734,85.41673,29-Apr-15,"Nankhel, Maligaun"
1,NaN,NaN,Medicine - rehydration,NaN,Medicine,unit,1.0,Undefined,#VALUE!,MISSION 29/4 *01,Bhaktapur,Nankhel,Maligaun,27.64220,85.45980,27.66734,85.41673,29-Apr-15,"Nankhel, Maligaun"
2,NaN,NaN,Piyush,NaN,Sanitation - Water purification,bottle,1.0,Undefined,#VALUE!,MISSION 29/4 *01,Bhaktapur,Nankhel,Maligaun,27.64220,85.45980,27.66734,85.41673,29-Apr-15,"Nankhel, Maligaun"
3,NaN,NaN,Tarp,NaN,Shelter,unit,8.0,Undefined,#VALUE!,MISSION 29/4 *01,Bhaktapur,Nankhel,Maligaun,27.64220,85.45980,27.66734,85.41673,29-Apr-15,"Nankhel, Maligaun"
4,NaN,NaN,Tarp,NaN,Shelter,unit,5.0,Undefined,#VALUE!,MISSION 29/4 *02,Kabhre Palanchok,BanepaN.P.,Aashapuri,27.63300,85.53020,27.58333,85.66667,29-Apr-15,"BanepaN.P., Aashapuri"
5,NaN,NaN,Tarp,NaN,Shelter,unit,4.0,Undefined,#VALUE!,MISSION 29/4 *03,Bhaktapur,Sipadol,Soche,27.64714,85.43797,27.66734,85.41673,29-Apr-15,"Sipadol, Soche"
6,NaN,NaN,Medicine - rehydration,NaN,Medicine,unit,20.0,Undefined,#VALUE!,MISSION 29/4 *04,Bhaktapur,Bhaktapur,Durlang,27.66734,85.41673,27.66734,85.41673,29-Apr-15,"Bhaktapur, Durlang"
7,NaN,NaN,Tarp,NaN,Shelter,unit,20.0,Undefined,#VALUE!,MISSION 29/4 *04,Bhaktapur,BhaktapurN.P.,Durlang,27.67480,85.43190,27.66734,85.41673,29-Apr-15,"BhaktapurN.P., Durlang"
8,NaN,NaN,Medical Kit,NaN,Medicine,unit,1.0,Undefined,#VALUE!,MISSION 29/4 *05,Bhaktapur,Sipadol,Naichal,27.64714,85.43797,27.66734,85.41673,29-Apr-15,"Sipadol, Naichal"
9,NaN,NaN,Tarp,NaN,Shelter,unit,7.0,Undefined,#VALUE!,MISSION 29/4 *05,Bhaktapur,Sipadol,Naichal,27.64714,85.43797,27.66734,85.41673,29-Apr-15,"Sipadol, Naichal"


In [62]:
df = df.drop(['FSC - KGs (sack=25, pack=1, box=4.5)', 'FSC - Total KGs'], axis =1)

In [63]:
df['Items'] = df['Items'].fillna(df['Items grouped'])

In [64]:
df = df.rename({'VDC Latitude':'lat', 'VDC Longitude': 'lon'}, axis=1)

In [65]:
df.drop(['Mission', 'Item group filter'], axis= 1)

,Items,Origin,Items grouped,Category,Unit,amount,District,VDC,Village/Ward/extra info,lat,lon,District Latitude,District Longitude,Date,Location name
0,Noodles,NaN,Noodles,Food - Dry Food,box,5.0,Bhaktapur,Nankhel,Maligaun,27.64220,85.45980,27.66734,85.41673,29-Apr-15,"Nankhel, Maligaun"
1,Medicine - rehydration,NaN,Medicine - rehydration,Medicine,unit,1.0,Bhaktapur,Nankhel,Maligaun,27.64220,85.45980,27.66734,85.41673,29-Apr-15,"Nankhel, Maligaun"
2,Piyush,NaN,Piyush,Sanitation - Water purification,bottle,1.0,Bhaktapur,Nankhel,Maligaun,27.64220,85.45980,27.66734,85.41673,29-Apr-15,"Nankhel, Maligaun"
3,Tarp,NaN,Tarp,Shelter,unit,8.0,Bhaktapur,Nankhel,Maligaun,27.64220,85.45980,27.66734,85.41673,29-Apr-15,"Nankhel, Maligaun"
4,Tarp,NaN,Tarp,Shelter,unit,5.0,Kabhre Palanchok,BanepaN.P.,Aashapuri,27.63300,85.53020,27.58333,85.66667,29-Apr-15,"BanepaN.P., Aashapuri"
5,Tarp,NaN,Tarp,Shelter,unit,4.0,Bhaktapur,Sipadol,Soche,27.64714,85.43797,27.66734,85.41673,29-Apr-15,"Sipadol, Soche"
6,Medicine - rehydration,NaN,Medicine - rehydration,Medicine,unit,20.0,Bhaktapur,Bhaktapur,Durlang,27.66734,85.41673,27.66734,85.41673,29-Apr-15,"Bhaktapur, Durlang"
7,Tarp,NaN,Tarp,Shelter,unit,20.0,Bhaktapur,BhaktapurN.P.,Durlang,27.67480,85.43190,27.66734,85.41673,29-Apr-15,"BhaktapurN.P., Durlang"
8,Medical Kit,NaN,Medical Kit,Medicine,unit,1.0,Bhaktapur,Sipadol,Naichal,27.64714,85.43797,27.66734,85.41673,29-Apr-15,"Sipadol, Naichal"
9,Tarp,NaN,Tarp,Shelter,unit,7.0,Bhaktapur,Sipadol,Naichal,27.64714,85.43797,27.66734,85.41673,29-Apr-15,"Sipadol, Naichal"


In [66]:
df['Date'] = pd.to_datetime(df['Date'])

In [67]:
df.to_csv('DB_ready.csv')

In [68]:
import requests
import csv
import json

In [69]:
# configuration values
config = dict(
    username="2660e8fa-aac2-45ef-81ba-217967728ad2-bluemix", 
    password="9cd1b00e6b6927e663bfa0e2e9d677e163378a7c98e09bfb598dd74a4439fc7a",
    database="cfc_patient_db")

# dict of table names and their csv representations
csv_lookup = {
    # "table_name": "path/to/file.csv"
    "Nepal": "DB_ready.csv"
}

# dict of request data, which we'll upload to Cloudant
requests_data = {}

for table, filepath in csv_lookup.items():
    request_data = dict(docs=[]) 
    # get our data
    with open(filepath, 'rt') as f:
        reader = csv.DictReader(f)
        # put into request body    
        for row in reader:
#             row['table_name'] = table # add doctype based on table
            request_data['docs'].append(row)
    requests_data[table] = request_data

#   Try this instead
requests_data = pd.DataFrame.to_json(df)
    

In [70]:
url = "https://{username}:{password}@{username}.cloudant.com/{database}/_bulk_docs".format(**config)

In [71]:
auth_headers = {"Content-Type": "application/x-www-form-urlencoded"}
r = requests.post(url= url, json=request_data)

In [72]:
r.text

'[{"ok":true,"id":"973fbfe2455a7419d02c24988ce33f39","rev":"1-9f99f11d26747145852150c58e5ddded"},{"ok":true,"id":"973fbfe2455a7419d02c24988ce3437e","rev":"1-b3834011c19314b04c077301fc8fdcdf"},{"ok":true,"id":"973fbfe2455a7419d02c24988ce3488d","rev":"1-0c69a54cf856b1fd6bbc8e445ae066bd"},{"ok":true,"id":"973fbfe2455a7419d02c24988ce35333","rev":"1-7bf4e04740d75773e9ce1e7f5de2a7a0"},{"ok":true,"id":"973fbfe2455a7419d02c24988ce36204","rev":"1-05a947af9e5e533519b1bbc5fcfc778f"},{"ok":true,"id":"973fbfe2455a7419d02c24988ce36a6c","rev":"1-727255f7655b1c314ee75e214632cb71"},{"ok":true,"id":"973fbfe2455a7419d02c24988ce3782d","rev":"1-f56f777854a30812162ad8c213b434ad"},{"ok":true,"id":"973fbfe2455a7419d02c24988ce38763","rev":"1-054e1f0faf5d346ceb51e6db005643a5"},{"ok":true,"id":"973fbfe2455a7419d02c24988ce38ea6","rev":"1-a82b06b5b320d12f6135d392a7661760"},{"ok":true,"id":"973fbfe2455a7419d02c24988ce38ede","rev":"1-19251f5f33e381a163167cf86d3be530"},{"ok":true,"id":"973fbfe2455a7419d02c24988ce39e7